# Fire Story Prototype
Hannah Catri 217569841

## Packages

In [1]:
#!pip install geopandas
#!pip install dash_core_components
#!pip install dash-html-components
#!pip install dash-table

In [4]:
import pandas as pd
from sodapy import Socrata
import datetime
import requests
import json
import geopandas as gpd
import plotly.express as px
import datetime
import dash
from dash import dash_table
#import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from sodapy import Socrata
import datetime
#from pandas_profiling import ProfileReport
import requests
import json
import geopandas as gpd
import dash
#import dash_html_components as html # depreciated
from dash import html
from dash import dcc
#import dash_core_components as dcc # depreciated
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output# Load Data

In [27]:
from dotenv import find_dotenv, load_dotenv
import os

# Loads the dotenv file into the environment. This exposes the variables in that
# file as though they were actual environment variables to the os module.
load_dotenv(find_dotenv())

BEARER_TOKEN = os.environ.get("BEARER_TOKEN")
MAP_API_KEY = os.environ.get("MAP_API_KEY")
MAPBOX_TOKEN= os.environ.get("MAPBOX_TOKEN")

## Data Connection and Validation

### Source: SF Open Data, Fire Dept Calls for Service

In [5]:
# set 10 day lookback window
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=10)).isoformat()
#Get count of incidents to determine limit to retrieve
num=pd.read_json("https://data.sfgov.org/resource/RowID.json?$select=COUNT(incident_number)&$where=call_date%20between%20%27"
                 +str(periodstart)+"%27%20AND%20%27"+str(periodend)+"%27")
maxNumber=num.iloc[0,0]
print('The number of incidents in the set is',maxNumber)
# Use record to write query call to API to get all incidents needed/bypass API default of 1000
query_str=("https://data.sfgov.org/resource/RowID.json?$where=call_date%20between%20%27"
           +str(periodstart)+"%27%20AND%20%27"
           +str(periodend)+"%27&$limit="+str(maxNumber))
cfs_data=pd.read_json(query_str)
print(cfs_data.shape)
cfs_data.sample(5)

The number of incidents in the set is 8670
(8670, 34)


,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,on_scene_dttm,transport_dttm,hospital_dttm
2909,220103259,78,22005071,Medical Incident,2022-01-10T00:00:00.000,2022-01-10T00:00:00.000,2022-01-10T21:58:20.000,2022-01-10T21:59:53.000,2022-01-10T22:00:06.000,2022-01-10T22:00:13.000,...,MEDIC,1,2,5,Western Addition,220103259-78,"{'type': 'Point', 'coordinates': [-122.4274691...",2022-01-10T22:04:44.000,2022-01-10T22:20:37.000,2022-01-10T22:30:57.000
1477,220081823,AM112,22003924,Medical Incident,2022-01-08T00:00:00.000,2022-01-08T00:00:00.000,2022-01-08T15:12:59.000,2022-01-08T15:13:31.000,2022-01-08T15:13:41.000,2022-01-08T15:14:14.000,...,PRIVATE,1,4,3,Russian Hill,220081823-AM112,"{'type': 'Point', 'coordinates': [-122.4194017...",NaN,NaN,NaN
2588,220090742,E41,22004233,Medical Incident,2022-01-09T00:00:00.000,2022-01-09T00:00:00.000,2022-01-09T09:21:12.000,2022-01-09T09:23:33.000,2022-01-09T09:23:52.000,NaN,...,ENGINE,2,4,3,Nob Hill,220090742-E41,"{'type': 'Point', 'coordinates': [-122.4201903...",NaN,NaN,NaN
5591,220120762,59,22005732,Medical Incident,2022-01-12T00:00:00.000,2022-01-12T00:00:00.000,2022-01-12T09:10:03.000,2022-01-12T09:11:24.000,2022-01-12T09:11:42.000,2022-01-12T09:11:45.000,...,MEDIC,2,5,5,Western Addition,220120762-59,"{'type': 'Point', 'coordinates': [-122.4327628...",2022-01-12T09:16:26.000,2022-01-12T09:38:08.000,2022-01-12T09:47:53.000
993,220070230,72,22003158,Medical Incident,2022-01-07T00:00:00.000,2022-01-06T00:00:00.000,2022-01-07T02:31:39.000,2022-01-07T02:33:30.000,2022-01-07T02:34:05.000,2022-01-07T02:34:11.000,...,MEDIC,2,9.0,10,Visitacion Valley,220070230-72,"{'type': 'Point', 'coordinates': [-122.4234232...",2022-01-07T02:44:08.000,2022-01-07T03:08:04.000,2022-01-07T03:29:10.000


### Source: Twitter, Keyword Retrieval

In [6]:
#bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMdDTQEAAAAAfYqWjo5%2BOgmFP%2FmqGGkq74pEahs%3DNqy7E6BdRittRWu2v3DJAVXEvbDLhAejdCBccR4mQPNO70s9Sl'

In [11]:
bearer_token = BEARER_TOKEN
# Twitter functions
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    
    return headers

def create_url(keyword,start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [12]:
periodend = datetime.datetime.now().isoformat()
periodstart = (datetime.datetime.now()- datetime.timedelta(days=2)).isoformat()
#Inputs for the request
headers = create_headers(bearer_token)
keyword = "san francisco fire lang:en"
#place = "San Francisco"
start_time = periodstart[:-2]+'Z'
end_time = periodend[:-2]+'Z'
max_results = 100

url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

tweets=pd.json_normalize(json_response['data'])
tweets.head()

Endpoint Response Code: 200


,reply_settings,author_id,source,conversation_id,created_at,text,lang,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets,in_reply_to_user_id
0,everyone,1425505436195713028,WordPress.com,1482804255220240385,2022-01-16T19:57:38.000Z,Sunday Links: San Francisco Fire Department Re...,en,1482804255220240385,0,0,0,0,NaN,NaN
1,everyone,1269612893529444354,WordPress.com,1482804251776663555,2022-01-16T19:57:37.000Z,Sunday Links: San Francisco Fire Department Re...,en,1482804251776663555,0,0,0,0,NaN,NaN
2,everyone,1283765688616628233,WordPress.com,1482802403745439744,2022-01-16T19:50:16.000Z,Sunday Links: San Francisco Fire Department Re...,en,1482802403745439744,0,0,0,0,NaN,NaN
3,everyone,1353332601482842114,WordPress.com,1482802398703857665,2022-01-16T19:50:15.000Z,Sunday Links: San Francisco Fire Department Re...,en,1482802398703857665,0,0,0,0,NaN,NaN
4,everyone,202551763,TweetDeck,1482797049141600256,2022-01-16T19:29:00.000Z,▶️ San Francisco 49ers @ Dallas Cowboys \n\nDa...,en,1482797049141600256,0,0,0,0,NaN,NaN


### Source: Weather Data, Historic and Forecast

Details: https://openweathermap.org/api/one-call-api

In [13]:
import requests
import json

lat=37.773972
lon=-122.431297
API_key=MAP_API_KEY
# see http://bulk.openweathermap.org/sample/ for ID lists
city_id='5391959'
#url=f"https://api.openweathermap.org/data/2.5/weather?id={city_id}&appid={API_key}"
url=f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=minutely&appid={API_key}&units=metric"
#weather=pd.read_json(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&appid={API_key}")
#weather=pd.read_json(f"https://api.openweathermap.org/data/2.5/weather?id={city_id}&appid={API_key}")
#print(weather.shape)
#weather.sample(5)


response = requests.get(url)
data = json.loads(response.text)
weather=pd.json_normalize(data)
hourly=pd.json_normalize(data['hourly'])
hourly.head()


,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop
0,1642392000,11.58,10.67,1017,72,6.71,0.0,75,10000,2.51,31,2.64,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
1,1642395600,11.74,10.82,1017,71,6.66,0.0,70,10000,2.52,32,2.66,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
2,1642399200,11.77,10.83,1017,70,6.48,0.0,63,10000,2.58,29,2.71,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
3,1642402800,11.67,10.72,1018,70,6.39,0.0,40,10000,2.79,29,3.09,"[{'id': 802, 'main': 'Clouds', 'description': ...",0
4,1642406400,11.48,10.54,1018,71,6.41,0.0,41,10000,2.86,30,3.12,"[{'id': 802, 'main': 'Clouds', 'description': ...",0


In [14]:
print(hourly.columns)

Index(['dt', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'uvi',
       'clouds', 'visibility', 'wind_speed', 'wind_deg', 'wind_gust',
       'weather', 'pop'],
      dtype='object')


## Data Fusion and Aggregation

In [15]:
cfs_data.head()

,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,on_scene_dttm,transport_dttm,hospital_dttm
0,220072357,72,22003499,Medical Incident,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T16:57:30.000,2022-01-07T16:59:31.000,2022-01-07T17:10:49.000,2022-01-07T17:10:52.000,...,MEDIC,2,8,7,West of Twin Peaks,220072357-72,"{'type': 'Point', 'coordinates': [-122.4676305...",NaN,NaN,NaN
1,220071636,T05,22003392,Medical Incident,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T13:07:20.000,2022-01-07T13:07:20.000,2022-01-07T13:08:01.000,NaN,...,TRUCK,3,5,5,Western Addition,220071636-T05,"{'type': 'Point', 'coordinates': [-122.4359746...",NaN,NaN,NaN
2,220071589,E41,22003382,Alarms,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T12:49:02.000,2022-01-07T12:50:57.000,2022-01-07T12:51:21.000,NaN,...,ENGINE,4,4,2,Pacific Heights,220071589-E41,"{'type': 'Point', 'coordinates': [-122.4262522...",NaN,NaN,NaN
3,220072323,E01,22003494,Medical Incident,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T16:46:39.000,2022-01-07T16:49:33.000,2022-01-07T17:18:08.000,2022-01-07T17:21:16.000,...,ENGINE,3,3,6,South of Market,220072323-E01,"{'type': 'Point', 'coordinates': [-122.4090260...",NaN,NaN,NaN
4,220072304,T01,22003491,Alarms,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T16:38:51.000,2022-01-07T16:40:42.000,2022-01-07T16:40:54.000,NaN,...,TRUCK,4,2,6,Tenderloin,220072304-T01,"{'type': 'Point', 'coordinates': [-122.4142160...",NaN,NaN,NaN


In [16]:
# Select relevant data needed for aggregation
cols=['dispatch_dttm','on_scene_dttm','battalion','original_priority','unit_type' ]
responseTimes=cfs_data[cols].copy()
# Clean data - remove empty values
responseTimes.dropna(inplace=True)
# Clean data - convert data to date/time
responseTimes['dispatch_dttm']= pd.to_datetime(responseTimes['dispatch_dttm'])
responseTimes['on_scene_dttm']= pd.to_datetime(responseTimes['on_scene_dttm'])
# Calculation - time from dispatch to on scene
responseTimes['ResponseMins']=((responseTimes.on_scene_dttm - responseTimes.dispatch_dttm).dt.total_seconds())/60
responseTimes.head(3)


,dispatch_dttm,on_scene_dttm,battalion,original_priority,unit_type,ResponseMins
5,2022-01-07 16:17:13,2022-01-07 16:19:42,B04,3,ENGINE,2.483333
6,2022-01-07 16:10:14,2022-01-07 16:20:22,B02,3,MEDIC,10.133333
8,2022-01-07 02:44:34,2022-01-07 02:49:24,B06,3,ENGINE,4.833333


In [18]:
test.dtypes

dispatch_dttm        datetime64[ns]
battalion                    object
original_priority            object
unit_type                    object
ResponseMins                float64
dispatch_time        datetime64[ns]
dtype: object

In [17]:
test = responseTimes.drop(columns=['on_scene_dttm'])
test['dispatch_time'] = test['dispatch_dttm'].apply(lambda x: x.replace(minute=0, second=0))
test

,dispatch_dttm,battalion,original_priority,unit_type,ResponseMins,dispatch_time
5,2022-01-07 16:17:13,B04,3,ENGINE,2.483333,2022-01-07 16:00:00
6,2022-01-07 16:10:14,B02,3,MEDIC,10.133333,2022-01-07 16:00:00
8,2022-01-07 02:44:34,B06,3,ENGINE,4.833333,2022-01-07 02:00:00
9,2022-01-07 00:57:29,B03,3,CHIEF,5.183333,2022-01-07 00:00:00
10,2022-01-07 00:03:36,B02,3,ENGINE,2.683333,2022-01-07 00:00:00
...,...,...,...,...,...,...
8664,2022-01-16 00:25:22,B02,3,ENGINE,6.016667,2022-01-16 00:00:00
8665,2022-01-16 01:14:05,B08,3,MEDIC,13.433333,2022-01-16 01:00:00
8667,2022-01-16 00:41:20,B02,3,ENGINE,0.000000,2022-01-16 00:00:00
8668,2022-01-16 00:53:39,B02,3,ENGINE,0.000000,2022-01-16 00:00:00


In [19]:
#test = test.drop(columns=['dispatch_dttm'])
dispatch_data_grouped = test.groupby(['dispatch_time','battalion','original_priority','unit_type']) \
                            .agg(min_sal = ('ResponseMins', 'min'), \
                                 mean_sal = ('ResponseMins', 'mean'),
                                 max_sal = ('ResponseMins', 'max')).round(2).reset_index()
#test=responseTimes.groupby('battalion').apply(lambda df: test.ResponseMins.mean())
dispatch_data_grouped

,dispatch_time,battalion,original_priority,unit_type,min_sal,mean_sal,max_sal
0,2022-01-07 00:00:00,B01,3,ENGINE,1.48,2.88,4.28
1,2022-01-07 00:00:00,B01,3,MEDIC,4.53,4.53,4.53
2,2022-01-07 00:00:00,B01,3,PRIVATE,6.27,6.27,6.27
3,2022-01-07 00:00:00,B02,3,ENGINE,2.68,2.68,2.68
4,2022-01-07 00:00:00,B02,3,TRUCK,1.50,1.50,1.50
...,...,...,...,...,...,...,...
5453,2022-01-16 02:00:00,B09,2,MEDIC,6.53,6.53,6.53
5454,2022-01-16 02:00:00,B10,2,ENGINE,6.32,6.32,6.32
5455,2022-01-16 02:00:00,B10,A,ENGINE,5.38,7.35,9.32
5456,2022-01-16 02:00:00,B10,A,MEDIC,7.63,7.63,7.63


In [20]:
# Old Version
#test = responseTimes.set_index('dispatch_dttm').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')
#test.head()
# New version
dispatch_data_grouped = test.groupby(['dispatch_time','battalion','original_priority','unit_type']) \
                            .agg(total_mins = ('ResponseMins', 'sum'), \
                                 num_records = ('ResponseMins', 'count')).round(2).reset_index()
#test=responseTimes.groupby('battalion').apply(lambda df: test.ResponseMins.mean())
dispatch_data_grouped

,dispatch_time,battalion,original_priority,unit_type,total_mins,num_records
0,2022-01-07 00:00:00,B01,3,ENGINE,5.77,2
1,2022-01-07 00:00:00,B01,3,MEDIC,4.53,1
2,2022-01-07 00:00:00,B01,3,PRIVATE,6.27,1
3,2022-01-07 00:00:00,B02,3,ENGINE,2.68,1
4,2022-01-07 00:00:00,B02,3,TRUCK,1.50,1
...,...,...,...,...,...,...
5453,2022-01-16 02:00:00,B09,2,MEDIC,6.53,1
5454,2022-01-16 02:00:00,B10,2,ENGINE,6.32,1
5455,2022-01-16 02:00:00,B10,A,ENGINE,14.70,2
5456,2022-01-16 02:00:00,B10,A,MEDIC,7.63,1


In [24]:
variable='battalion'
#fig = px.line(dispatch_data_grouped, x='dispatch_time', y=variable, markers=True)
fig = px.line(dispatch_data_grouped, x='dispatch_time', y=variable, markers=True)
#facet_col="company", facet_col_wrap=2
fig.show()

In [25]:
variable='temp'
fig = px.line(hourly, x='dt', y=variable, markers=True)
fig.show()

In [10]:
# Aggregation - Response time by day
df = responseTimes.set_index('dispatch_dttm').groupby(pd.Grouper(freq='d')).mean().dropna(how='all')
df.head(3)

,ResponseMins
dispatch_dttm,
2021-12-30,8.132887
2021-12-31,7.596259
2022-01-01,7.403230


In [16]:
#Exclude all retweets
tweets.drop(tweets[tweets['referenced_tweets'].notna()].index, inplace=True)
tweets.reset_index(inplace=True, drop=True)
tweets.head()

,reply_settings,conversation_id,source,created_at,id,author_id,text,lang,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,in_reply_to_user_id,referenced_tweets,geo.coordinates.type,geo.coordinates.coordinates,geo.place_id
0,everyone,1479727655465279492,AcidRemap Index Processor Real,2022-01-08T08:12:19.000Z,1479727655465279492,376919041,"Updated: San Francisco County, CA; Sacramento ...",en,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,everyone,1479610607708037120,WordPress.com,2022-01-08T00:27:13.000Z,1479610607708037120,1388527425001189376,Deadly Santa Barbara Submarine Fire Brings New...,en,0,0,1,0,NaN,NaN,NaN,NaN,NaN
2,everyone,1479580418898862080,WordPress.com,2022-01-07T22:27:15.000Z,1479580418898862080,1388527425001189376,Firefighters help elderly woman escape Rohnert...,en,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,everyone,1479575128593747968,Campaign Share,2022-01-07T22:06:14.000Z,1479575128593747968,634012094,The underinsurance of homes in the San Francis...,en,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,everyone,1479546422500286469,Twitter for iPhone,2022-01-07T20:12:10.000Z,1479546422500286469,1301990125925093381,Public safety agencies in the U.S. are experie...,en,3,0,14,0,NaN,NaN,NaN,NaN,NaN


In [17]:
# Select relevant data needed for twitter scroll
cols=['text','created_at' ]
skinnyTweets=tweets[cols].copy()
skinnyTweets.head()

,text,created_at
0,"Updated: San Francisco County, CA; Sacramento ...",2022-01-08T08:12:19.000Z
1,Deadly Santa Barbara Submarine Fire Brings New...,2022-01-08T00:27:13.000Z
2,Firefighters help elderly woman escape Rohnert...,2022-01-07T22:27:15.000Z
3,The underinsurance of homes in the San Francis...,2022-01-07T22:06:14.000Z
4,Public safety agencies in the U.S. are experie...,2022-01-07T20:12:10.000Z


In [18]:
skinnyTweets=skinnyTweets.rename(columns={"text": "Tweet", "created_at": "Timestamp"})

## Data Visuals

In [26]:
fig = px.line(df,title = 'Avg Response Time Past Week')
# GREY = AGGREGATE DATA
fig.update_layout(paper_bgcolor="#e0e0e0",
                 font_color="black",
                 xaxis_title="Date (Prior 10 days)",
                  yaxis_title="Average Response Time (Minutes)")

fig.show()

NameError: name 'df' is not defined

In [20]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(skinnyTweets.columns),
                fill_color='#F28C28',
                align='left'),
    cells=dict(values=[skinnyTweets.Tweet,skinnyTweets.Timestamp],
               fill_color='#FAC898',
               align='left'))
])

fig.show()

In [21]:
hourly['dt'] = pd.to_datetime(hourly['dt'],unit='s')
hourly.head()

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop
0,2022-01-08 19:00:00,10.68,9.94,1021,82,7.74,1.87,75,10000,3.69,64,5.44,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
1,2022-01-08 20:00:00,10.91,10.15,1021,80,7.60,2.18,62,10000,3.52,53,5.33,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
2,2022-01-08 21:00:00,11.31,10.53,1021,78,7.62,2.03,48,10000,3.69,37,4.86,"[{'id': 802, 'main': 'Clouds', 'description': ...",0
3,2022-01-08 22:00:00,11.75,10.96,1020,76,7.66,1.45,34,10000,3.98,39,4.97,"[{'id': 802, 'main': 'Clouds', 'description': ...",0
4,2022-01-08 23:00:00,12.25,11.44,1020,73,7.56,0.77,19,10000,3.82,41,5.01,"[{'id': 801, 'main': 'Clouds', 'description': ...",0


In [22]:
variable='temp'
fig = px.line(hourly, x='dt', y=variable, markers=True)
fig.show()

In [23]:
climate_features=['temp','feels_like','pressure','humidity','dew_point','uvi','clouds','visibility','wind_speed','wind_deg','wind_gust']

In [1]:

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(id='climate_dropdown',
                value='temp',
                options=[{'label':ctype, 'value':ctype}
                for ctype in climate_features]),
    dcc.Graph(id='weatherChart')
])
@app.callback(Output('weatherChart','figure'),
             Input ('climate_dropdown','value'))
def plot_iclimate_features(ctype):
    fig=go.Figure()
    fig = px.line(hourly, x='dt', y=ctype, markers=True)
    fig.layout.title=f'Weather data for {ctype}'
    fig.layout.paper_bgcolor="#FFD700"
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline',host='127.0.0.1', port=8053, debug=True)

NameError: name 'JupyterDash' is not defined

In [28]:
cfs_data.sample(5)

,call_number,unit_id,incident_number,call_type,call_date,watch_date,received_dttm,entry_dttm,dispatch_dttm,response_dttm,...,number_of_alarms,unit_type,unit_sequence_in_call_dispatch,fire_prevention_district,supervisor_district,neighborhoods_analysis_boundaries,rowid,case_location,transport_dttm,hospital_dttm
8281,220071791,96,22003415,Medical Incident,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T13:58:31.000,2022-01-07T13:59:02.000,2022-01-07T14:09:21.000,2022-01-07T14:11:39.000,...,1,MEDIC,3,2,8,Castro/Upper Market,220071791-96,"{'type': 'Point', 'coordinates': [-122.4295303...",2022-01-07T14:48:39.000,2022-01-07T14:51:00.000
5689,220040680,KM206,22001644,Medical Incident,2022-01-04T00:00:00.000,2022-01-04T00:00:00.000,2022-01-04T08:52:35.000,2022-01-04T08:54:20.000,2022-01-04T08:54:41.000,2022-01-04T08:55:19.000,...,1,PRIVATE,1,7,1,Inner Richmond,220040680-KM206,"{'type': 'Point', 'coordinates': [-122.4653508...",2022-01-04T09:42:42.000,2022-01-04T09:49:34.000
7020,220063242,94,22003087,Medical Incident,2022-01-06T00:00:00.000,2022-01-06T00:00:00.000,2022-01-06T21:35:07.000,2022-01-06T21:36:56.000,2022-01-06T21:37:12.000,2022-01-06T21:37:25.000,...,1,MEDIC,3,9,10,Visitacion Valley,220063242-94,"{'type': 'Point', 'coordinates': [-122.4171412...",NaN,NaN
8909,220071228,T03,22003321,Medical Incident,2022-01-07T00:00:00.000,2022-01-07T00:00:00.000,2022-01-07T11:09:48.000,2022-01-07T11:11:36.000,2022-01-07T11:13:06.000,2022-01-07T11:14:40.000,...,1,TRUCK,1,4,3,Nob Hill,220071228-T03,"{'type': 'Point', 'coordinates': [-122.4194923...",NaN,NaN
1400,213652043,72,21162231,Medical Incident,2021-12-31T00:00:00.000,2021-12-31T00:00:00.000,2021-12-31T16:36:11.000,2021-12-31T16:36:11.000,2021-12-31T16:36:32.000,2021-12-31T16:36:43.000,...,1,MEDIC,4,4,2,Marina,213652043-72,"{'type': 'Point', 'coordinates': [-122.4333916...",NaN,NaN


In [31]:


incoming_calls = cfs_data.sort_values(["call_date"], ascending=False)
fig = go.Figure(data=[go.Table(
    header=dict(values=['Call Type','Suburb','Timestamp'],
                fill_color='#F28C28',
                align='left'),
    cells=dict(values=[incoming_calls.call_type,incoming_calls.neighborhoods_analysis_boundaries, incoming_calls.call_date],
               fill_color='#FAC898',
               align='left'))
])

fig.show()

In [29]:
geo_df = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

px.set_mapbox_access_token(MAPBOX_TOKEN)
fig = px.scatter_geo(geo_df,
                    lat=geo_df.geometry.y,
                    lon=geo_df.geometry.x,
                    hover_name="name")
fig.show()
